In [ ]:
# Imports
import numpy as np
import pandas as pd

In [ ]:
data = pd.concat([pd.read_csv('Data/2009.csv'), pd.read_csv('Data/2010.csv'), 
                  pd.read_csv('Data/2011.csv'), pd.read_csv('Data/2012.csv'), 
                  pd.read_csv('Data/2013.csv'), pd.read_csv('Data/2014.csv'), 
                  pd.read_csv('Data/2015.csv'), pd.read_csv('Data/2016.csv'), 
                  pd.read_csv('Data/2017.csv'), pd.read_csv('Data/2018.csv')])

In [ ]:
# Convert string to DateTime
data.FL_DATE = pd.to_datetime(data.FL_DATE, infer_datetime_format=True)


In [ ]:
# Drop NaN values in the variables we will use in our baseline model
data = data[data['TAXI_OUT'].notna()]
data = data[data['ORIGIN'].notna()]
data = data[data['DEST'].notna()]

In [ ]:
# Compare shape to see effect of preprocessing
print('Shape before:',data.shape,'\n Shape after:',df.shape)

In [ ]:
# Save as new dataframe
data.to_csv('Data/data_all.csv',index=False)